# Unlicensed omeprazole liquids

**Aim:**  
Use dm+d available fields to reliably differentiate unlicensed vs licensed omeprazole liquid VMPs.  

We want to:
- Find VMPs where all associated AMPs are unlicensed  
- Find VMPs where all associated AMPs are licensed  
- Identify VMPs with mixed licensed/unlicensed products  


### Setting Up Our Tools

In [10]:
# Import the libraries we need:
from ebmdatalab import bq                  # Connects us to the prescribing database
import pandas as pd                        # Our main tool for handling data tables
import matplotlib.pyplot as plt            # Creates charts and graphs
import numpy as np                         # Helps with numerical calculations
import os                                  # Manages file paths
from datetime import datetime              # For date handling
import seaborn as sns                      # For enhanced visualisations
import os
os.environ["EBMDATALAB_BQ_CREDENTIALS_PATH"] = "../bq-service-account.json"


# This setting ensures we can see full product names in our results
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

### Availability restriction column analysis

AMP table contains a column 'avail_restrict'. This uses a code to detail availability of products. The table 'availabilityrestriction' details the meaning behind these codes.

In [11]:
sql = '''
SELECT * FROM dmd.availabilityrestriction
'''
avail_rest_DF = bq.cached_read(sql, csv_path=os.path.join('..','data','avail_rest.csv'))
avail_rest_DF = avail_rest_DF.fillna('None')
avail_rest_DF

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,cd,descr
0,1,None
1,2,Restricted Availability
2,3,Individual Patient Supply
3,4,Imported
4,5,Clinical Trial
5,6,Special
6,7,Extemp
7,9,Not available
8,10,For Patient Safety Consideration


Special products can therefore be identified at AMP level with 'avail_restrict' == 6

### Identifying products

We can use VTM and oral form route to select the appropriate products.

In [30]:
sql = """
SELECT
    vtm.nm as vtm_name,
    vmp.id AS vmp_id,
    vmp.nm AS vmp_name,
    amp.id AS amp_id,
    amp.nm AS amp_name,
    amp.avail_restrict,
    ofr.descr
FROM dmd.vmp vmp
INNER JOIN dmd.vtm vtm ON vtm.id = vmp.vtm
INNER JOIN dmd.ont ont ON ont.vmp = vmp.id
INNER JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
INNER JOIN dmd.amp amp ON vmp.id = amp.vmp
WHERE vtm.id = 776991000 -- omeprazole VTM
AND ofr.descr IN ('solution.oral', 'solution.gastroenteral', 'suspension.oral', 'suspension.gastroenteral');
"""

# Execute query and cache results
df = bq.cached_read(sql, csv_path=os.path.join('..', 'data', 'omep_specials.csv'))
with pd.option_context('display.max_rows', 120):
    display(df)

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,vtm_name,vmp_id,vmp_name,amp_id,amp_name,avail_restrict,descr
0,Omeprazole,39021411000001102,Omeprazole 10mg dispersible gastro-resistant tablets,10544411000001102,Losec MUPS 10mg gastro-resistant tablets,9,suspension.oral
1,Omeprazole,39021511000001103,Omeprazole 20mg dispersible gastro-resistant tablets,10835311000001108,Losec MUPS 20mg gastro-resistant tablets,9,suspension.oral
2,Omeprazole,39021611000001104,Omeprazole 40mg dispersible gastro-resistant tablets,11385311000001108,Losec MUPS 40mg gastro-resistant tablets,9,suspension.oral
3,Omeprazole,39021411000001102,Omeprazole 10mg dispersible gastro-resistant tablets,13175311000001100,Losec MUPS 10mg gastro-resistant tablets,9,suspension.oral
4,Omeprazole,39021511000001103,Omeprazole 20mg dispersible gastro-resistant tablets,13175511000001106,Losec MUPS 20mg gastro-resistant tablets,9,suspension.oral
5,Omeprazole,39021411000001102,Omeprazole 10mg dispersible gastro-resistant tablets,13926511000001100,Losec MUPS 10mg gastro-resistant tablets,9,suspension.oral
6,Omeprazole,39021511000001103,Omeprazole 20mg dispersible gastro-resistant tablets,13926711000001105,Losec MUPS 20mg gastro-resistant tablets,9,suspension.oral
7,Omeprazole,39021411000001102,Omeprazole 10mg dispersible gastro-resistant tablets,14336711000001103,Losec MUPS 10mg gastro-resistant tablets,1,suspension.oral
8,Omeprazole,39021511000001103,Omeprazole 20mg dispersible gastro-resistant tablets,14366011000001109,Losec MUPS 20mg gastro-resistant tablets,1,suspension.oral
9,Omeprazole,39021611000001104,Omeprazole 40mg dispersible gastro-resistant tablets,14367211000001102,Losec MUPS 40mg gastro-resistant tablets,1,suspension.oral


This gives some unexpected results with some dispersible gastro-resistant tablets having an oral form route of suspension.oral. We need to try and filter these out.

In [31]:
sql = """
SELECT
    vtm.nm as vtm_name,
    vmp.id AS vmp_id,
    vmp.nm AS vmp_name,
    amp.id AS amp_id,
    amp.nm AS amp_name,
    amp.avail_restrict,
    ofr.descr
FROM dmd.vmp vmp
INNER JOIN dmd.vtm vtm ON vtm.id = vmp.vtm
INNER JOIN dmd.ont ont ON ont.vmp = vmp.id
INNER JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
INNER JOIN dmd.amp amp ON vmp.id = amp.vmp
WHERE amp.id = 10544411000001102 -- AMP Losec MUPS 10mg gastro-resistant tablets
OR amp.id = 10835311000001108 -- AMP Losec MUPS 20mg gastro-resistant tablets;
"""

# Execute query and cache results
df = bq.cached_read(sql, csv_path=os.path.join('..', 'data', 'omep_disp_tab.csv'))
df

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,vtm_name,vmp_id,vmp_name,amp_id,amp_name,avail_restrict,descr
0,Omeprazole,39021411000001102,Omeprazole 10mg dispersible gastro-resistant tablets,10544411000001102,Losec MUPS 10mg gastro-resistant tablets,9,tablet.oral
1,Omeprazole,39021411000001102,Omeprazole 10mg dispersible gastro-resistant tablets,10544411000001102,Losec MUPS 10mg gastro-resistant tablets,9,suspension.oral
2,Omeprazole,39021511000001103,Omeprazole 20mg dispersible gastro-resistant tablets,10835311000001108,Losec MUPS 20mg gastro-resistant tablets,9,tablet.oral
3,Omeprazole,39021511000001103,Omeprazole 20mg dispersible gastro-resistant tablets,10835311000001108,Losec MUPS 20mg gastro-resistant tablets,9,suspension.oral


Here we can see that these products have 2 matching oral form route descriptions - tablet.oral and suspension.oral. We therefore need to filter out any products with a tablet.oral description.

In [27]:
sql = """
    SELECT 
        vmp.id,
        vmp.nm
    FROM dmd.vmp vmp
    LEFT JOIN dmd.vtm vtm ON vmp.vtm = vtm.id
    LEFT JOIN dmd.ont ont ON ont.vmp = vmp.id
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    WHERE vtm.id = 776991000
    AND ofr.descr = 'tablet.oral'
"""

# Execute query and cache results
df = bq.cached_read(sql, csv_path=os.path.join('..', 'data', 'exclude_disp_tabs.csv'))
df

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,id,nm
0,39021411000001102,Omeprazole 10mg dispersible gastro-resistant tablets
1,39021511000001103,Omeprazole 20mg dispersible gastro-resistant tablets
2,39021611000001104,Omeprazole 40mg dispersible gastro-resistant tablets


In [32]:
sql = """
SELECT
    vtm.nm as vtm_name,
    vmp.id AS vmp_id,
    vmp.nm AS vmp_name,
    amp.id AS amp_id,
    amp.nm AS amp_name,
    amp.avail_restrict,
    ofr.descr
FROM dmd.vmp vmp
INNER JOIN dmd.vtm vtm ON vtm.id = vmp.vtm
INNER JOIN dmd.ont ont ON ont.vmp = vmp.id
INNER JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
INNER JOIN dmd.amp amp ON vmp.id = amp.vmp
WHERE vtm.id = 776991000 -- omeprazole VTM
AND ofr.descr IN ('solution.oral', 'solution.gastroenteral', 'suspension.oral', 'suspension.gastroenteral')
AND vmp.id NOT IN (
    SELECT 
        vmp.id
    FROM dmd.vmp vmp
    LEFT JOIN dmd.vtm vtm ON vmp.vtm = vtm.id
    LEFT JOIN dmd.ont ont ON ont.vmp = vmp.id
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    WHERE vtm.id = 776991000
    AND ofr.descr = 'tablet.oral'
);
"""

# Execute query and cache results
df = bq.cached_read(sql, csv_path=os.path.join('..', 'data', 'omep_specials_exc_disp_tabs.csv'))
with pd.option_context('display.max_rows', 120):
    display(df)

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,vtm_name,vmp_id,vmp_name,amp_id,amp_name,avail_restrict,descr
0,Omeprazole,8670511000001102,Omeprazole 10mg/5ml oral suspension,8643311000001103,Omeprazole 10mg/5ml oral suspension,6,suspension.oral
1,Omeprazole,8670611000001103,Omeprazole 2.5mg/5ml oral suspension,8643811000001107,Omeprazole 2.5mg/5ml oral suspension,6,suspension.oral
2,Omeprazole,8670711000001107,Omeprazole 20mg/5ml oral suspension,8644211000001109,Omeprazole 20mg/5ml oral suspension,6,suspension.oral
3,Omeprazole,8670811000001104,Omeprazole 5mg/5ml oral suspension,8644711000001102,Omeprazole 5mg/5ml oral suspension,6,suspension.oral
4,Omeprazole,13000911000001107,Omeprazole 12.5mg/5ml oral suspension,12958811000001105,Omeprazole 12.5mg/5ml oral suspension,6,suspension.oral
5,Omeprazole,13000811000001102,Omeprazole 12.5mg/5ml oral solution,12959711000001106,Omeprazole 12.5mg/5ml oral solution,6,solution.oral
6,Omeprazole,13001111000001103,Omeprazole 15.5mg/5ml oral suspension,12960911000001109,Omeprazole 15.5mg/5ml oral suspension,6,suspension.oral
7,Omeprazole,13001011000001104,Omeprazole 15.5mg/5ml oral solution,12961711000001104,Omeprazole 15.5mg/5ml oral solution,6,solution.oral
8,Omeprazole,13001311000001101,Omeprazole 25mg/5ml oral suspension,12963111000001104,Omeprazole 25mg/5ml oral suspension,6,suspension.oral
9,Omeprazole,13001211000001109,Omeprazole 25mg/5ml oral solution,12965311000001101,Omeprazole 25mg/5ml oral solution,6,solution.oral


### Summarise avail_restrict per VMP

SCMD only provides data at a VMP level. We need to investigate if VMPs contain a mixture of special and non-special products.

In [34]:
sql = """
SELECT
    vtm.nm as vtm_name,
    vmp.id AS vmp_id,
    vmp.nm AS vmp_name,
    SUM(CASE WHEN amp.avail_restrict = 6 THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN amp.avail_restrict != 6 THEN 1 ELSE 0 END) AS non_special_count
FROM dmd.vmp vmp
INNER JOIN dmd.vtm vtm ON vtm.id = vmp.vtm
INNER JOIN dmd.ont ont ON ont.vmp = vmp.id
INNER JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
INNER JOIN dmd.amp amp ON vmp.id = amp.vmp
WHERE vtm.id = 776991000 -- omeprazole VTM
AND ofr.descr IN ('solution.oral', 'solution.gastroenteral', 'suspension.oral', 'suspension.gastroenteral')
AND vmp.id NOT IN (
    SELECT 
        vmp.id
    FROM dmd.vmp vmp
    LEFT JOIN dmd.vtm vtm ON vmp.vtm = vtm.id
    LEFT JOIN dmd.ont ont ON ont.vmp = vmp.id
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    WHERE vtm.id = 776991000
    AND ofr.descr = 'tablet.oral'
)
GROUP BY
    vtm.nm,
    vmp.id,
    vmp.nm;
"""

df = bq.cached_read(sql, csv_path=os.path.join('..', 'data', 'special_counts.csv'))
df

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,vtm_name,vmp_id,vmp_name,special_count,non_special_count
0,Omeprazole,8670511000001102,Omeprazole 10mg/5ml oral suspension,1,0
1,Omeprazole,8670611000001103,Omeprazole 2.5mg/5ml oral suspension,1,0
2,Omeprazole,8670711000001107,Omeprazole 20mg/5ml oral suspension,1,0
3,Omeprazole,8670811000001104,Omeprazole 5mg/5ml oral suspension,1,0
4,Omeprazole,13000911000001107,Omeprazole 12.5mg/5ml oral suspension,1,0
5,Omeprazole,13000811000001102,Omeprazole 12.5mg/5ml oral solution,1,0
6,Omeprazole,13001111000001103,Omeprazole 15.5mg/5ml oral suspension,1,0
7,Omeprazole,13001011000001104,Omeprazole 15.5mg/5ml oral solution,1,0
8,Omeprazole,13001311000001101,Omeprazole 25mg/5ml oral suspension,1,0
9,Omeprazole,13001211000001109,Omeprazole 25mg/5ml oral solution,1,0


We can see that the all the VMPs containing special AMPs do not contain any licensed AMPs. So we can reliably identify specials products at a VMP level for omeprazole liquids.

In [36]:
sql = """
SELECT
    vtm.nm as vtm_name,
    vmp.id AS vmp_id,
    vmp.nm AS vmp_name,
    SUM(CASE WHEN amp.avail_restrict = 6 THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN amp.avail_restrict != 6 THEN 1 ELSE 0 END) AS non_special_count
FROM dmd.vmp vmp
INNER JOIN dmd.vtm vtm ON vtm.id = vmp.vtm
INNER JOIN dmd.ont ont ON ont.vmp = vmp.id
INNER JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
INNER JOIN dmd.amp amp ON vmp.id = amp.vmp
WHERE vtm.id = 776991000 -- omeprazole VTM
AND ofr.descr IN ('solution.oral', 'solution.gastroenteral', 'suspension.oral', 'suspension.gastroenteral')
AND vmp.id NOT IN (
    SELECT 
        vmp.id
    FROM dmd.vmp vmp
    LEFT JOIN dmd.vtm vtm ON vmp.vtm = vtm.id
    LEFT JOIN dmd.ont ont ON ont.vmp = vmp.id
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    WHERE vtm.id = 776991000
    AND ofr.descr = 'tablet.oral'
)
GROUP BY
    vtm.nm,
    vmp.id,
    vmp.nm
HAVING 
    special_count > 0
    AND
    non_special_count = 0;
"""

df = bq.cached_read(sql, csv_path=os.path.join('..', 'data', 'omep_specials_only.csv'))
df

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,vtm_name,vmp_id,vmp_name,special_count,non_special_count
0,Omeprazole,8670511000001102,Omeprazole 10mg/5ml oral suspension,1,0
1,Omeprazole,8670611000001103,Omeprazole 2.5mg/5ml oral suspension,1,0
2,Omeprazole,8670711000001107,Omeprazole 20mg/5ml oral suspension,1,0
3,Omeprazole,8670811000001104,Omeprazole 5mg/5ml oral suspension,1,0
4,Omeprazole,13000911000001107,Omeprazole 12.5mg/5ml oral suspension,1,0
5,Omeprazole,13000811000001102,Omeprazole 12.5mg/5ml oral solution,1,0
6,Omeprazole,13001111000001103,Omeprazole 15.5mg/5ml oral suspension,1,0
7,Omeprazole,13001011000001104,Omeprazole 15.5mg/5ml oral solution,1,0
8,Omeprazole,13001311000001101,Omeprazole 25mg/5ml oral suspension,1,0
9,Omeprazole,13001211000001109,Omeprazole 25mg/5ml oral solution,1,0
